In [1]:
import torch
import torchvision 
import torch.nn.functional as F  

import torchvision.transforms as transforms
from torch import optim  # For optimizers SGD, Adam, etc.
from torch import nn  # All neural network modules
from torch.utils.data import DataLoader
from tqdm import tqdm # For nice progress bar!

In [2]:
target_device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
import pandas as pd

In [4]:
import os

In [6]:
import matplotlib.pyplot as plt
import numpy as np
import cv2 # pip install opencv-python

# Normally, we put pictures in a dir
# And we have a file that contains img_name and # label or etc info, easier and more efficient to load data

In [7]:
### root_dir saves the dir of pictures
### csv_file saves the file dir of imgs(name) and labels
### transform is a function, sometimes we use it for data augmentation 
### or normalization(std mean....)
from torch.utils.data import Dataset
from torch.utils.data import DataLoader 
##success from Dataset
class my_dataset(Dataset):
    def __init__(self, csv_file, root_dir, transform):
        self.annotations = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
    #def __init__(self , csv_file , root_dir , transform=None):
    #    ### annotation is a map from img name to label
    #    self.annotations = pd.read_csv(csv_file)
    #    ##self.length = len(self.annotations)
        
        self.root_dir = root_dir
        self.transform = transform
    
    ##define len(self.annotations)
    def __len__(self):
        return len(self.annotations) 
    
    def __getitem__(self, index):
        ### index is the idex of pictures
        ### !!!! .iloc[] is a function in panda 
        img_path = os.path.join(self.root_dir ,self.annotations.iloc[index, 0] )
        image = cv2.imread(img_path)
        y_label = torch.tensor(int(self.annotations.iloc[index, 1]))
        
        ### in this code, this just put img to tensor, more usage to be continued.
        if self.transform:
            image = self.transform(image)

        return (image, y_label)
    
dataset = my_dataset(
csv_file="cats_dogs.csv",
root_dir="cats_dogs_resized",
transform=transforms.ToTensor()
)

In [8]:
# Hyperparameters
in_channel = 3
num_classes = 2
learning_rate = 1e-3
batch_size = 32
num_epochs = 10

In [9]:
# Dataset is actually a lot larger ~25k images, just took out 10 pictures
# to upload to Github. It's enough to understand the structure and scale
# if you got more images.
train_set, test_set = torch.utils.data.random_split(dataset, [5, 5])
train_loader = DataLoader(dataset=train_set, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_set, batch_size=batch_size, shuffle=True)

# Model
model = torchvision.models.googlenet(pretrained=True)
model.to(target_device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Train Network
for epoch in range(num_epochs):
    losses = []

    for batch_idx, (data, targets) in enumerate(train_loader):
        # Get data to cuda if possible
        data = data.to(device=target_device)
        targets = targets.to(device=target_device)

        # forward
        scores = model(data)
        loss = criterion(scores, targets)

        losses.append(loss.item())

        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradient descent or adam step
        optimizer.step()

    print(f"Cost at epoch {epoch} is {sum(losses)/len(losses)}")

# Check accuracy on training to see how good our model is
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=target_device)
            y = y.to(device=target_device)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

        print(
            f"Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}"
        )

    model.train()


print("Checking accuracy on Training Set")
check_accuracy(train_loader, model)

print("Checking accuracy on Test Set")
check_accuracy(test_loader, model)

Cost at epoch 0 is 7.773490905761719
Cost at epoch 1 is 4.837826728820801
Cost at epoch 2 is 2.6733341217041016
Cost at epoch 3 is 1.199251651763916
Cost at epoch 4 is 0.8784807324409485
Cost at epoch 5 is 0.12971068918704987
Cost at epoch 6 is 0.0762890875339508
Cost at epoch 7 is 0.06263137608766556
Cost at epoch 8 is 0.03211470693349838
Cost at epoch 9 is 0.020281733945012093
Checking accuracy on Training Set
Got 5 / 5 with accuracy 100.00
Checking accuracy on Test Set
Got 3 / 5 with accuracy 60.00
